In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [17]:
df = spark.read.csv('test2.csv',header=True,inferSchema=True)

In [10]:
df.show()

+-----+----+----------+-------+
| Name| Age|Experience| Salary|
+-----+----+----------+-------+
|  ALI|  25|        10| 5000.0|
|JACOB|  56|         5|40000.0|
| JOHN|  12|         8|30000.0|
| null|  30|         9|10000.0|
| null|null|      null|90000.0|
| null|null|      null|   null|
+-----+----+----------+-------+



In [5]:
df.drop('Name').show()

+---+----------+-------+
|Age|Experience| Salary|
+---+----------+-------+
| 25|        10| 5000.0|
| 56|         5|40000.0|
| 12|         8|30000.0|
+---+----------+-------+



In [11]:
#drop the whole row if any cell is null
df.na.drop().show()

+-----+---+----------+-------+
| Name|Age|Experience| Salary|
+-----+---+----------+-------+
|  ALI| 25|        10| 5000.0|
|JACOB| 56|         5|40000.0|
| JOHN| 12|         8|30000.0|
+-----+---+----------+-------+



In [14]:
#drop if all cells of a row is null
df.na.drop(how='all').show()
#drop the row if it got less than two none null cells
df.na.drop(how='all',thresh=2).show()

+-----+----+----------+-------+
| Name| Age|Experience| Salary|
+-----+----+----------+-------+
|  ALI|  25|        10| 5000.0|
|JACOB|  56|         5|40000.0|
| JOHN|  12|         8|30000.0|
| null|  30|         9|10000.0|
| null|null|      null|90000.0|
+-----+----+----------+-------+

+-----+---+----------+-------+
| Name|Age|Experience| Salary|
+-----+---+----------+-------+
|  ALI| 25|        10| 5000.0|
|JACOB| 56|         5|40000.0|
| JOHN| 12|         8|30000.0|
| null| 30|         9|10000.0|
+-----+---+----------+-------+



In [15]:
#if na is in a certain columns drop it
df.na.drop(how="any",subset=['Age']).show()

+-----+---+----------+-------+
| Name|Age|Experience| Salary|
+-----+---+----------+-------+
|  ALI| 25|        10| 5000.0|
|JACOB| 56|         5|40000.0|
| JOHN| 12|         8|30000.0|
| null| 30|         9|10000.0|
+-----+---+----------+-------+



In [18]:
#fill nas
df.na.fill('Missing Values').show()

+--------------+----+----------+-------+
|          Name| Age|Experience| Salary|
+--------------+----+----------+-------+
|           ALI|  25|        10| 5000.0|
|         JACOB|  56|         5|40000.0|
|          JOHN|  12|         8|30000.0|
|Missing Values|  30|         9|10000.0|
|Missing Values|null|      null|90000.0|
|Missing Values|null|      null|   null|
+--------------+----+----------+-------+



In [21]:
#specific column
df.na.fill('Missing Values','Experience').show()
#multiple columns
df.na.fill('Missing Values',['Experience','Age']).show()

+-----+----+----------+-------+
| Name| Age|Experience| Salary|
+-----+----+----------+-------+
|  ALI|  25|        10| 5000.0|
|JACOB|  56|         5|40000.0|
| JOHN|  12|         8|30000.0|
| null|  30|         9|10000.0|
| null|null|      null|90000.0|
| null|null|      null|   null|
+-----+----+----------+-------+

+-----+----+----------+-------+
| Name| Age|Experience| Salary|
+-----+----+----------+-------+
|  ALI|  25|        10| 5000.0|
|JACOB|  56|         5|40000.0|
| JOHN|  12|         8|30000.0|
| null|  30|         9|10000.0|
| null|null|      null|90000.0|
| null|null|      null|   null|
+-----+----+----------+-------+



In [31]:
#filling the na using imputer
from pyspark.ml.feature import Imputer
imputer = Imputer(
inputCols = ['Age','Experience','Salary'],
    outputCols = ["{}_imputed".format(c) for c in ['Age','Experience','Salary']]).setStrategy('mean')
imputer.fit(df).transform(df).show()

+-----+----+----------+-------+-----------+------------------+--------------+
| Name| Age|Experience| Salary|Age_imputed|Experience_imputed|Salary_imputed|
+-----+----+----------+-------+-----------+------------------+--------------+
|  ALI|  25|        10| 5000.0|         25|                10|        5000.0|
|JACOB|  56|         5|40000.0|         56|                 5|       40000.0|
| JOHN|  12|         8|30000.0|         12|                 8|       30000.0|
| null|  30|         9|10000.0|         30|                 9|       10000.0|
| null|null|      null|90000.0|         30|                 8|       90000.0|
| null|null|      null|   null|         30|                 8|       35000.0|
+-----+----+----------+-------+-----------+------------------+--------------+



### filters

In [39]:
df.filter('Salary <= 20000').show()
#or
df.filter(df['Salary']<=20000).show()
#this is the inverse or not of the above
df.filter(~(df['Salary']<=20000)).show()

+----+---+----------+-------+
|Name|Age|Experience| Salary|
+----+---+----------+-------+
| ALI| 25|        10| 5000.0|
|null| 30|         9|10000.0|
+----+---+----------+-------+

+----+---+----------+-------+
|Name|Age|Experience| Salary|
+----+---+----------+-------+
| ALI| 25|        10| 5000.0|
|null| 30|         9|10000.0|
+----+---+----------+-------+

+-----+----+----------+-------+
| Name| Age|Experience| Salary|
+-----+----+----------+-------+
|JACOB|  56|         5|40000.0|
| JOHN|  12|         8|30000.0|
| null|null|      null|90000.0|
+-----+----+----------+-------+



In [33]:
#show only two columns
df.filter('Salary <= 20000').select('Name','Age').show()

+----+---+
|Name|Age|
+----+---+
| ALI| 25|
|null| 30|
+----+---+



In [34]:
#two conditions
df.filter(df['Salary']<=20000).show()

+----+---+----------+-------+
|Name|Age|Experience| Salary|
+----+---+----------+-------+
| ALI| 25|        10| 5000.0|
|null| 30|         9|10000.0|
+----+---+----------+-------+



In [38]:
df.filter((df['Salary']<=20000) & (df['Age']>=30)).show()

+----+---+----------+-------+
|Name|Age|Experience| Salary|
+----+---+----------+-------+
|null| 30|         9|10000.0|
+----+---+----------+-------+



In [42]:
df2 = spark.read.csv('test3.csv',header=True,inferSchema=True)
df2.show()

+-----+----+----------+-------+----------+
| Name| Age|Experience| Salary|Department|
+-----+----+----------+-------+----------+
|  ALI|  25|        10| 5000.0|    physic|
|JACOB|  56|         5|40000.0|   geology|
| JOHN|  12|         8|30000.0|    physic|
| null|  30|         9|10000.0|   geology|
| null|null|      null|90000.0|      data|
| null|null|      null|   null|    device|
|  ALI|  25|        10| 5000.0|    physic|
|JACOB|  56|         5|40000.0|   geology|
| JOHN|  12|         8|30000.0|    physic|
| null|  30|         9|10000.0|   geology|
| null|null|      null|90000.0|      data|
| null|null|      null|   null|    device|
+-----+----+----------+-------+----------+



In [45]:
#groupby, meaning sum the salary, age, and expereince of all ALIs
df2.groupby('Name').sum().show()

+-----+--------+---------------+-----------+
| Name|sum(Age)|sum(Experience)|sum(Salary)|
+-----+--------+---------------+-----------+
|  ALI|      50|             20|    10000.0|
| null|      60|             18|   200000.0|
| JOHN|      24|             16|    60000.0|
|JACOB|     112|             10|    80000.0|
+-----+--------+---------------+-----------+



In [47]:
#groub all the departments
df2.groupby('Department').sum().show()
df2.groupby('Department').mean().show()
df2.groupby('Department').count().show()


+----------+--------+---------------+-----------+
|Department|sum(Age)|sum(Experience)|sum(Salary)|
+----------+--------+---------------+-----------+
|    physic|      74|             36|    70000.0|
|    device|    null|           null|       null|
|      data|    null|           null|   180000.0|
|   geology|     172|             28|   100000.0|
+----------+--------+---------------+-----------+

+----------+--------+---------------+-----------+
|Department|avg(Age)|avg(Experience)|avg(Salary)|
+----------+--------+---------------+-----------+
|    physic|    18.5|            9.0|    17500.0|
|    device|    null|           null|       null|
|      data|    null|           null|    90000.0|
|   geology|    43.0|            7.0|    25000.0|
+----------+--------+---------------+-----------+

+----------+-----+
|Department|count|
+----------+-----+
|    physic|    4|
|    device|    2|
|      data|    2|
|   geology|    4|
+----------+-----+



In [49]:
#you can also do the groupby like this
df2.agg({'Salary':'mean'}).show()

+-----------+
|avg(Salary)|
+-----------+
|    35000.0|
+-----------+



In [51]:
#show those who get the highest salary
df2.groupby('Name').max().show()

+-----+--------+---------------+-----------+
| Name|max(Age)|max(Experience)|max(Salary)|
+-----+--------+---------------+-----------+
|  ALI|      25|             10|     5000.0|
| null|      30|              9|    90000.0|
| JOHN|      12|              8|    30000.0|
|JACOB|      56|              5|    40000.0|
+-----+--------+---------------+-----------+

